<a href="https://colab.research.google.com/github/pragneshrana/Algo_Trial/blob/master/MultipleRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
'''
Code to extract the OLS summary of different parameter.
TO save in excel file
RUN model and append the result
'''
import statsmodels.api as sm
import numpy as np

Y = [1,3,4,5,2,3,4]
X = range(1,8)
X = sm.add_constant(X)
model = sm.OLS(Y,X)
results = model.fit()
# print('Summary:',results.summary())
# print('params:',results.params)
# print('tvalues:',results.tvalues)
# print('ttest:',results.t_test([1, 0]))
# print('ftest:',results.f_test(np.identity(2)))
print('Conition Number:',results.condition_number)
print('Eigen values:',results.eigenvals)
print('Rsquared:',results.rsquared)
print('F-statistic:', results.fvalue)
print('Prob (F-statistic):', results.f_pvalue)
print('Log-Likelihood:', None)
print('AIC:', results.aic)
print('BIC:', results.bic)
print('SSR:',results.ssr)
print('Confidence intervasl:',results.conf_int())
print('Number of observations :',results.nobs)

Conition Number: 10.40388203202207
Eigen values: [145.65434845   1.34565155]
Rsquared: 0.16118421052631593
F-statistic: 0.9607843137254913
Prob (F-statistic): 0.37200486130017757
Log-Likelihood: None
AIC: 25.70718166636078
BIC: 25.599001964471405
SSR: 9.107142857142856
Confidence intervasl: [[-0.78920715  5.07492144]
 [-0.40562951  0.90562951]]
Number of observations : 7.0
